In [1]:
#| default_exp 08_sandwich-with-switched-roles-for-wikiseealsotitles

In [2]:
#| hide
from nbdev.showdoc import *
import nbdev; nbdev.nbdev_export()

In [3]:
%reload_ext autoreload
%autoreload 2

In [4]:
#| export
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '0,1'

import torch,json, torch.multiprocessing as mp, joblib, numpy as np, scipy.sparse as sp

from xcai.basics import *
from xcai.models.sandwich import SAW002, SandwichConfig

In [5]:
os.environ['WANDB_MODE'] = 'disabled'

In [6]:
#| export
os.environ['WANDB_PROJECT'] = 'sandwich_00-wikiseealsotitles'

## Setup

In [7]:
output_dir = '/scratch/scai/phd/aiz218323/outputs/cachew/06_ngame-for-wikiseealsotitles-with-augmented-training'

data_dir = '/home/scai/phd/aiz218323/scratch/datasets/benchmarks/'
config_file = 'wikiseealsotitles'
config_key = 'data_meta'

mname = 'sentence-transformers/msmarco-distilbert-base-v4'

In [8]:
pickle_dir = '/scratch/scai/phd/aiz218323/datasets/processed/'

In [9]:
use_sxc_sampler = True

pkl_file = f'{pickle_dir}/sandwich/wikiseealsotitles_data-meta_distilbert-base-uncased'
pkl_file = f'{pkl_file}_sxc' if use_sxc_sampler else f'{pkl_file}_xcs'
pkl_file = f'{pkl_file}.joblib'

In [10]:
os.makedirs(os.path.dirname(pkl_file), exist_ok=True)
block = build_block(pkl_file, config_file, use_sxc_sampler, config_key, do_build=False, only_test=False, 
                    n_slbl_samples=1, main_oversample=False, n_sdata_meta_samples=1, n_slbl_meta_samples=1, meta_oversample=False,
                    data_dir=data_dir)

In [85]:
#| export
from typing import Dict, Optional

from xcai.graph.operations import *
from xcai.sdata import *

In [86]:
#| export
def fill_matrix(matrix:torch.Tensor, matrix_info:Dict, filler:torch.Tensor, filler_info:Dict, matrix_meta:Optional[torch.Tensor]=None,
                filler_meta:Optional[torch.Tensor]=None):
    row_idx = np.where(matrix.getnnz(axis=1) == 0)[0]
    col_idx = np.where(filler[row_idx].getnnz(axis=0) > 0)[0]

    mat = filler[:, col_idx]
    mat_info = {}
    for k,v in filler_info.items():
        if isinstance(v, torch.Tensor): mat_info[k] = v[col_idx]
        else: mat_info[k] = type(v)([v[i] for i in col_idx])

    indices, indptr = [], np.zeros(matrix.shape[0]+1, dtype=np.int64)
    for idx in row_idx:
        p,q = mat.indptr[idx], mat.indptr[idx+1]
        i = mat.indices[p:q]
        indices.append(i)
        indptr[idx+1] = len(i)
    indices = np.hstack(indices)
    indptr = indptr.cumsum()

    mat = sp.csr_matrix((np.ones_like(indices, dtype=np.float32), indices, indptr))
    
    comb_matrix = sp.hstack([matrix, mat])
    comb_info = SXCDataset.combine_info(matrix_info, mat_info)

    if matrix_meta is not None:
        filler_meta = filler_meta[col_idx]
        comb_matrix_meta = sp.vstack([matrix_meta, filler_meta])
        return comb_matrix, comb_info, comb_matrix_meta

    return comb_matrix, comb_info
    

In [87]:
#| export
def swap_lbl_and_meta(dset, meta_name:str, thresh:Optional[int]=20, batch_size:Optional[int]=1024, topk:Optional[int]=10, 
                      perform_aug:Optional[bool]=True, fill_empty:Optional[bool]=True):
    data_lbl = Graph.threshold_on_degree(dset.data.data_lbl, thresh=thresh)
    _, lbl_lbl = Graph.one_hop_matrix(data_lbl, batch_size=batch_size, topk=topk, do_normalize=False)

    m_dset = dset.meta[f'{meta_name}_meta']
    data_meta = sp.vstack([dset.data.data_lbl, lbl_lbl]) if perform_aug else dset.data.data_lbl
    lbl_meta = m_dset.lbl_meta.transpose().tocsr()

    data_lbl, data_info, lbl_info = m_dset.data_meta, dset.data.data_info, m_dset.meta_info
    if perform_aug: data_lbl, data_info = sp.vstack([data_lbl, m_dset.lbl_meta]), SXCDataset.combine_info(data_info, dset.data.lbl_info)

    if fill_empty: data_lbl, lbl_info, lbl_meta = fill_matrix(data_lbl, lbl_info, data_meta, dset.data.lbl_info, lbl_meta, lbl_lbl)
    
    meta_dset = SMetaXCDataset(prefix='main', data_meta=data_meta, lbl_meta=lbl_meta, meta_info=dset.data.lbl_info, 
                               n_data_meta_samples=m_dset.n_data_meta_samples, n_lbl_meta_samples=m_dset.n_lbl_meta_samples,
                               meta_info_keys=m_dset.meta_info_keys, n_sdata_meta_samples=m_dset.n_sdata_meta_samples, 
                               n_slbl_meta_samples=m_dset.n_slbl_meta_samples, meta_oversample=m_dset.meta_oversample, 
                               use_meta_distribution=m_dset.use_meta_distribution)

    data_lbl_filterer = None
    if not fill_empty:
        x_idx = np.where(data_lbl.getnnz(axis=1) == 0)[0].reshape(-1,1)
        y_idx = np.zeros((len(x_idx),1), dtype=np.int64)
        data_lbl[x_idx, y_idx] = 1
        data_lbl_filterer = np.hstack([x_idx, y_idx])

    data_dset = SMainXCDataset(data_info=data_info, data_lbl=data_lbl, lbl_info=lbl_info, data_lbl_filterer=data_lbl_filterer,
                               n_lbl_samples=dset.data.n_lbl_samples, data_info_keys=dset.data.data_info_keys, 
                               lbl_info_keys=dset.data.lbl_info_keys, n_slbl_samples=dset.data.n_slbl_samples, 
                               main_oversample=dset.data.main_oversample, use_main_distribution=dset.data.use_main_distribution)

    return SXCDataset(data_dset, **{'main_meta': meta_dset})
    

In [69]:
meta_name = 'cat'

In [52]:
train_dset = swap_lbl_and_meta(block.train.dset, meta_name, thresh=20, batch_size=1024, topk=10)

  0%|          | 0/306 [00:00<?, ?it/s]

  0%|          | 0/677 [00:00<?, ?it/s]

In [79]:
train_dset.data.data_lbl

<Compressed Sparse Row sparse matrix of dtype 'float32'
	with 4925830 stored elements and shape (1005412, 832893)>

In [80]:
[len(v) for v in train_dset.data.data_info.values()]

[1005412, 1005412, 1005412, 1005412]

In [81]:
[len(v) for v in train_dset.data.lbl_info.values()]

[832893, 832893, 832893, 832893]

In [82]:
train_dset.meta['main_meta'].data_meta

<Compressed Sparse Row sparse matrix of dtype 'float32'
	with 3278885 stored elements and shape (1005412, 312330)>

In [83]:
train_dset.meta['main_meta'].lbl_meta

<Compressed Sparse Row sparse matrix of dtype 'float32'
	with 2334548 stored elements and shape (832893, 312330)>

In [84]:
[len(v) for v in train_dset.meta['main_meta'].meta_info.values()]

[312330, 312330, 312330, 312330]

In [70]:
test_dset = swap_lbl_and_meta(block.test.dset, meta_name, thresh=20, batch_size=1024, topk=10, perform_aug=False, fill_empty=False)

  0%|          | 0/306 [00:00<?, ?it/s]

  0%|          | 0/174 [00:00<?, ?it/s]

In [73]:
test_dset.data.data_lbl

<Compressed Sparse Row sparse matrix of dtype 'float32'
	with 834856 stored elements and shape (177515, 656086)>

In [74]:
[len(v) for v in test_dset.data.data_info.values()]

[177515, 177515, 177515, 177515]

In [75]:
[len(v) for v in test_dset.data.lbl_info.values()]

[656086, 656086, 656086, 656086]

In [76]:
test_dset.meta['main_meta'].data_meta

<Compressed Sparse Row sparse matrix of dtype 'float32'
	with 540833 stored elements and shape (177515, 312330)>

In [77]:
test_dset.meta['main_meta'].lbl_meta

<Compressed Sparse Row sparse matrix of dtype 'float32'
	with 1126099 stored elements and shape (656086, 312330)>

In [78]:
[len(v) for v in test_dset.meta['main_meta'].meta_info.values()]

[312330, 312330, 312330, 312330]

## Driver

In [ ]:
#| export
if __name__ == '__main__':
    output_dir = '/scratch/scai/phd/aiz218323/outputs/cachew/08_sandwich-with-switched-roles-for-wikiseealsotitles'

    data_dir = '/home/scai/phd/aiz218323/scratch/datasets/benchmarks/'
    config_file = 'wikiseealsotitles'
    config_key = 'data_meta'

    meta_name = 'cat'
    mname = 'sentence-transformers/msmarco-distilbert-base-v4'

    input_args = parse_args()

    pkl_file = f'{input_args.pickle_dir}/sandwich/wikiseealsotitles_data-meta_distilbert-base-uncased'
    pkl_file = f'{pkl_file}_sxc' if input_args.use_sxc_sampler else f'{pkl_file}_xcs'
    if input_args.only_test: pkl_file = f'{pkl_file}_only-test'
    pkl_file = f'{pkl_file}.joblib'

    do_inference = input_args.do_train_inference or input_args.do_test_inference or input_args.save_train_prediction or input_args.save_test_prediction or input_args.save_representation

    os.makedirs(os.path.dirname(pkl_file), exist_ok=True)
    block = build_block(pkl_file, config_file, input_args.use_sxc_sampler, config_key, do_build=input_args.build_block, only_test=input_args.only_test, 
                        n_slbl_samples=1, main_oversample=False, n_sdata_meta_samples=1, n_slbl_meta_samples=1, meta_oversample=False,
                        data_dir=data_dir)

    args = XCLearningArguments(
        output_dir=output_dir,
        logging_first_step=True,
        per_device_train_batch_size=512,
        per_device_eval_batch_size=512,
        representation_num_beams=200,
        representation_accumulation_steps=10,
        save_strategy="steps",
        eval_strategy="steps",
        eval_steps=5000,
        save_steps=5000,
        save_total_limit=5,
        num_train_epochs=300,
        predict_with_representation=True,
        adam_epsilon=1e-6,
        warmup_steps=100,
        weight_decay=0.01,
        learning_rate=2e-4,
        representation_search_type='BRUTEFORCE',
    
        output_representation_attribute='data_enriched_repr',
        label_representation_attribute='data_enriced_repr',
        representation_attribute='data_enriched_repr',
        clustering_representation_attribute='data_enriched_repr',
        metadata_representation_attribute='data_repr',
        data_augmentation_attribute='data_repr',

        group_by_cluster=True,
        num_clustering_warmup_epochs=10,
        num_cluster_update_epochs=5,
        num_cluster_size_update_epochs=25,
        use_data_metadata_for_clustering=True,
        clustering_type='EXPO',
        minimum_cluster_size=2,
        maximum_cluster_size=1600,

        metric_for_best_model='P@1',
        load_best_model_at_end=True,
        target_indices_key='plbl2data_idx',
        target_pointer_key='plbl2data_data2ptr',
    
        use_distributional_representation=False,
        use_encoder_parallel=True,
        max_grad_norm=None,
        bf16 = True,
    
        label_names=[f'{meta_name}2data_input_ids', f'{meta_name}2data_attention_mask', f'{meta_name}2data_idx', 
                     f'{meta_name}2data_data2ptr', f'p{meta_name}2data_idx', f'p{meta_name}2data_data2ptr',
                     f'{meta_name}2lbl_input_ids', f'{meta_name}2lbl_attention_mask', f'{meta_name}2lbl_idx', 
                     f'{meta_name}2lbl_lbl2ptr', f'p{meta_name}2lbl_idx', f'p{meta_name}2lbl_lbl2ptr'],

        data_aug_prefix=meta_name,
        use_label_metadata=True,
                     
        prune_metadata=False,
        num_metadata_prune_warmup_epochs=10,
        num_metadata_prune_epochs=5,
        metadata_prune_batch_size=2048,
        prune_metadata_names=[f'{meta_name}_meta'],
        use_data_metadata_for_pruning=True,
    
        predict_with_augmentation=False,
        use_augmentation_index_representation=True,

        data_aug_meta_name=meta_name,
        augmentation_num_beams=None,
        
        data_meta_batch_size=2048,
        augment_metadata=False,
        num_metadata_augment_warmup_epochs=10,
        num_metadata_augment_epochs=5,
    
        use_cpu_for_searching=True,
        use_cpu_for_clustering=True,
    )

    bsz = max(args.per_device_train_batch_size, args.per_device_eval_batch_size)*torch.cuda.device_count()
    
    config = SandwichConfig(
        data_aug_meta_prefix='cat2data', 
        lbl2data_aug_meta_prefix='cat2lbl',
    
        data_enrich=True,
        lbl2data_enrich=True,
    
        batch_size=bsz, 
        num_batch_labels=5000, 
        margin=0.3,
        num_negatives=10,
        tau=0.1,
        apply_softmax=True,
    
        use_calib_loss=True,
        calib_loss_weight=0.1,
        calib_margin=0.05,
        calib_num_negatives=10,
        calib_tau=0.1,
        calib_apply_softmax=False,
    
        use_query_loss=True,
    
        use_meta_loss=True,
        meta_loss_weight=1.0,
        
        use_encoder_parallel=True
    )
    
    def model_fn(mname):
        model = SAW002.from_pretrained(mname, config=config)
        return model
    
    def init_fn(model):
        model.init_heads_to_identity()
        model.init_meta_distilbert()
        model.init_combiner_to_last_layer()

    metric = PrecReclMrr(block.n_lbl, block.test.data_lbl_filterer, prop=block.train.dset.data.data_lbl,
                         pk=10, rk=200, rep_pk=[1, 3, 5, 10], rep_rk=[10, 100, 200], mk=[5, 10, 20])
    
    model = load_model(args.output_dir, model_fn, {"mname": mname}, init_fn, do_inference=do_inference, 
                       use_pretrained=input_args.use_pretrained)

    train_dset = swap_lbl_and_meta(block.train.dset, meta_name, thresh=20, batch_size=1024, topk=10)
    test_dset = swap_lbl_and_meta(block.test.dset, meta_name, thresh=20, batch_size=1024, topk=10, perform_aug=False, fill_empty=False)
    
    learn = XCLearner(
        model=model,
        args=args,
        train_dataset=train_dset,
        eval_dataset=test_dset,
        data_collator=block.collator,
        compute_metrics=metric,
    )
    
    main(learn, input_args, n_lbl=block.n_lbl)
    